In [ ]:
#!conda install pyshp

All data cleaned and put into a useable format using ArcGIS. I can write up documentatin for that later

In [1]:
#import shapefile 
import numpy as np
import pandas as pd
import sklearn.model_selection
import sklearn.linear_model
import sklearn.decomposition

blargh write some documentation

In [2]:
predictors = pd.read_csv('pnw_predictors.txt')
carbon = pd.read_csv('pnw_c.txt')

In [3]:
full_data = pd.merge(predictors, carbon, how='left', left_on=['X','Y'], right_on=['X', 'Y'])
or_sequestration = full_data[['raster_OR_c']].dropna()['raster_OR_c']
wa_sequestration = full_data[['raster_WA_c']].dropna()['raster_WA_c']
full_sequestration = list(or_sequestration)+list(wa_sequestration)
full_data['raster_OR_c'].fillna(full_data['raster_WA_c'], inplace=True)

cleaner = full_data.drop(['X', 'Y', 'OBJECTID_x', 'OR_random_Merge_x', 'OBJECTID_y', 'OR_random_Merge_y', 'raster_WA_c'], axis=1)
cleaner = cleaner.dropna()

In [20]:
pca = sklearn.decomposition.PCA(n_components=5)
pca.fit(cleaner.iloc[:,0:23])
small = pca.transform(cleaner.iloc[:,0:23])
train_pca = pd.DataFrame(small)

In [25]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train_pca, cleaner.iloc[:,23], test_size=0.2)

In [37]:
model = sklearn.linear_model.LinearRegression()
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.125888351404851

so dimensionality reduction of the predictor variables didn't help...

In [38]:
X_train2, X_test2, y_train2, y_test2= sklearn.model_selection.train_test_split(
cleaner.loc[:,['Bare_ground_change_1982_2016__1_','Short_vegetation_change_1982_2016','Tree_canopy_change_1982_2016']], cleaner.iloc[:,23], test_size=0.2)

In [41]:
model = sklearn.linear_model.LinearRegression()
model.fit(X_train2, y_train2)
model.score(X_train2, y_train2)

0.030974498587849175

and climate was at least somewhat more useful than just the ground change data